In [1]:
 # Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/02-Homework_04-Pandas_Instructions_PyCitySchools_Resources_schools_complete.csv"
student_data_to_load = "Resources/02-Homework_04-Pandas_Instructions_PyCitySchools_Resources_students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load).rename(columns={'School ID': 'school_id'})
student_data = pd.read_csv(student_data_to_load).rename(columns={'Student ID': 'student_id'})
print(school_data.columns)
print(student_data.columns)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete = school_data_complete.rename(columns={'Student ID': 'student_id', 'School ID': 'school_id'})
school_data_complete.head()

Index(['school_id', 'school_name', 'type', 'size', 'budget'], dtype='object')
Index(['student_id', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score'],
      dtype='object')


,student_id,student_name,gender,grade,school_name,reading_score,math_score,school_id,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [2]:
total_schools = len(school_data_complete['school_id'].unique())
total_students = len(school_data_complete['student_id'])
total_budget = school_data['budget'].sum()
math_average = school_data_complete['math_score'].mean()
reading_average = school_data_complete['reading_score'].mean()

passing_math = school_data_complete.loc[school_data_complete['math_score'] > 69]
percentage_passing_math = (len(passing_math) / total_students) * 100

passing_reading = school_data_complete.loc[school_data_complete['reading_score'] > 69]
percentage_passing_reading = (len(passing_reading) / total_students) * 100

passing = np.array([percentage_passing_math, percentage_passing_reading])
overall_passing_rate = np.mean(passing)

summary = pd.DataFrame({'Total Schools': [total_schools], 'Total Students': [total_students], 'Total Budget': [total_budget], 'Average Math Score': [math_average], 'Average Reading Score': [reading_average], '% Passing Math': [percentage_passing_math], '% Passing Reading': [percentage_passing_reading], '% Overall Passing Rate': [overall_passing_rate]})
summary.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.393158


In [3]:
school_grouping = school_data_complete.groupby('school_name').agg({'type': 'min', 'student_id':'count', 'budget':'min', 'math_score': 'mean', 'reading_score': 'mean'})
school_grouping = school_grouping.rename(columns={'type': 'School Type', 'student_id': 'Total Students', 'budget': 'Total School Budget', 'math_score' : 'Average Math Score', 'reading_score': 'Average Reading Score'})
school_grouping['Per Student Budget'] = school_grouping['Total School Budget'] / school_grouping['Total Students']

passing_math = school_data_complete.copy().loc[school_data_complete['math_score'] > 69].groupby('school_name').agg({'student_name' : 'count'})
passing_math = passing_math.rename(columns={'student_name' : '% Passing Math'})

passing_reading = school_data_complete.copy().loc[school_data_complete['reading_score'] > 69].groupby('school_name').agg({'student_name' : 'count'})
passing_reading = passing_reading.rename(columns={'student_name' : '% Passing Reading'})

school_grouping = school_grouping.join(passing_math, on='school_name')
school_grouping = school_grouping.join(passing_reading, on='school_name')
school_grouping['% Passing Reading'] = (school_grouping['% Passing Reading'] / school_grouping['Total Students']) * 100
school_grouping['% Passing Math'] = (school_grouping['% Passing Math'] / school_grouping['Total Students']) *100
school_grouping['% Overall Passing Rate'] = (school_grouping['% Passing Reading'] + school_grouping['% Passing Math']) / 2
school_grouping = school_grouping.sort_values(by=['% Overall Passing Rate'], ascending=False)
best_school_grouping = school_grouping.head(5).copy()
best_school_grouping['Average Math Score'] =  best_school_grouping['Average Math Score'].map("{:.2f}%".format)
best_school_grouping['Average Reading Score'] =  best_school_grouping['Average Reading Score'].map("{:.2f}%".format)
best_school_grouping['% Passing Math'] =  best_school_grouping['% Passing Math'].map("{:.2f}%".format)
best_school_grouping['% Passing Reading'] =  best_school_grouping['% Passing Reading'].map("{:.2f}%".format)
best_school_grouping['% Overall Passing Rate'] =  best_school_grouping['% Overall Passing Rate'].map("{:.2f}%".format)

best_school_grouping


,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,83.06%,83.98%,582.0,94.13%,97.04%,95.59%
Thomas High School,Charter,1635,1043130,83.42%,83.85%,638.0,93.27%,97.31%,95.29%
Pena High School,Charter,962,585858,83.84%,84.04%,609.0,94.59%,95.95%,95.27%
Griffin High School,Charter,1468,917500,83.35%,83.82%,625.0,93.39%,97.14%,95.27%
Wilson High School,Charter,2283,1319574,83.27%,83.99%,578.0,93.87%,96.54%,95.20%


In [4]:
worst_school_grouping = school_grouping.copy()
worst_school_grouping = worst_school_grouping.sort_values(by=['% Overall Passing Rate'])
worst_school_grouping['Average Math Score'] =  worst_school_grouping['Average Math Score'].map("{:.2f}%".format)
worst_school_grouping['Average Reading Score'] =  worst_school_grouping['Average Reading Score'].map("{:.2f}%".format)
worst_school_grouping['% Passing Math'] =  worst_school_grouping['% Passing Math'].map("{:.2f}%".format)
worst_school_grouping['% Passing Reading'] =  worst_school_grouping['% Passing Reading'].map("{:.2f}%".format)
worst_school_grouping['% Overall Passing Rate'] =  worst_school_grouping['% Overall Passing Rate'].map("{:.2f}%".format)
worst_school_grouping.head(5)

,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,76.84%,80.74%,637.0,66.37%,80.22%,73.29%
Figueroa High School,District,2949,1884411,76.71%,81.16%,639.0,65.99%,80.74%,73.36%
Huang High School,District,2917,1910635,76.63%,81.18%,655.0,65.68%,81.32%,73.50%
Johnson High School,District,4761,3094650,77.07%,80.97%,650.0,66.06%,81.22%,73.64%
Ford High School,District,2739,1763916,77.10%,80.75%,644.0,68.31%,79.30%,73.80%


In [5]:
math_by_grades = school_data_complete.copy()
math_by_grades = math_by_grades.groupby(['school_name', 'grade']).agg({'math_score': 'mean'}).unstack(level=1)
math_by_grades.columns = math_by_grades.columns.droplevel()
math_by_grades = math_by_grades[['9th', '10th', '11th', '12th']]
math_by_grades

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [6]:
reading_by_grades = school_data_complete.copy()
reading_by_grades = reading_by_grades.groupby(['school_name', 'grade']).agg({'reading_score': 'mean'}).unstack(level=1)
reading_by_grades.columns = reading_by_grades.columns.droplevel()
reading_by_grades = reading_by_grades[['9th', '10th', '11th', '12th']]
reading_by_grades

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [7]:
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
school_spending = school_grouping.copy()
school_spending['Per Student Budget'] = school_spending['Total School Budget'] / school_spending['Total Students']
school_spending = school_spending[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate', 'Per Student Budget']]
school_spending['Budget Group'] = pd.cut(school_spending['Per Student Budget'], spending_bins, labels = group_names)
school_spending = school_spending.groupby('Budget Group').agg({ 'Average Math Score' : 'mean', 'Average Reading Score' : 'mean', '% Passing Math' : 'mean', '% Passing Reading' : 'mean', '% Overall Passing Rate' : 'mean'})
school_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Budget Group,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


In [8]:
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_size = school_grouping.copy()
school_size = school_size[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate', 'Total Students']]
school_size['School Size'] = pd.cut(school_size['Total Students'], size_bins, labels = group_names)
school_size = school_size.groupby('School Size').agg({ 'Average Math Score' : 'mean', 'Average Reading Score' : 'mean', '% Passing Math' : 'mean', '% Passing Reading' : 'mean', '% Overall Passing Rate' : 'mean'})
school_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


In [9]:
school_type = school_grouping.copy()
school_type = school_type[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate', 'School Type']]
school_type = school_type.groupby('School Type').agg({ 'Average Math Score' : 'mean', 'Average Reading Score' : 'mean', '% Passing Math' : 'mean', '% Passing Reading' : 'mean', '% Overall Passing Rate' : 'mean'})
school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
